In [ ]:
# read "financial" csv
financial = pd.read_csv(r"C:\Users\Vanessa\Downloads\Projeto Final\2016 - 2023.csv", usecols = [0, 1, 2, 4, 5, 7], index_col = None)

In [ ]:
# split csv into a list a dataframes
financial_list = np.split(financial, financial[(financial.isna().all(axis = 1))|(financial.iloc[:,0].str.contains('MÊS'))].index)

In [ ]:
# identify not null dataframes from the list, rename columns and add them to another list
clean_data = []
for i in financial_list:
    if i.isna().values.all():
        pass
    else:
        if i.iloc[:,0].str.contains('MÊS').any():
            i.columns = ['dia', 'receita', 'depositante', 'despesas', 'favorecido', 'saldo']
            clean_data.append(i)     
clean_data

In [ ]:
# add "month" and "year" columns, reorder columns, drop unwanted rows and save dataframes as excel files
for i in range(len(clean_data)):
    clean_data[i]['mês'] = re.findall('\w+', clean_data[i].iloc[0,1])[0]
    clean_data[i]['ano'] = '20' + (re.findall('\w+', clean_data[i].iloc[0,1])[1]) 
    clean_data[i] = clean_data[i][['ano', 'mês', 'dia', 'receita', 'depositante', 'despesas', 'favorecido', 'saldo']]
    clean_data[i].drop(clean_data[i].index[0:3], inplace = True)
    clean_data[i].to_excel(f'tabela_{i}.xlsx', index = False)

# Data reading and cleaning

In [3]:
# import libraries and read csv
import pandas as pd
import numpy as np
import re
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

In [4]:
# read excel files after manual cleaning and join tables in a single dataset
tables_df = pd.DataFrame()
for i in range(0,82):
    a = pd.read_excel(fr"C:\Users\Vanessa\Downloads\Dados\Programas\Repositório Git\NGO_DataAnalysis\clean tables\tabela_{i}.xls")
    tables_df = pd.concat([tables_df, a], ignore_index = True)
tables_df.dtypes

ano            float64
mês             object
dia            float64
receita         object
depositante     object
despesas        object
favorecido      object
saldo           object
dtype: object

In [5]:
# create "expenses" dataframe and drop nulls
expenses = tables_df[['ano', 'mês', 'dia', 'despesas', 'favorecido']].copy()
expenses.dropna(subset = 'despesas', axis = 0, inplace=True)
expenses.reset_index(drop = True, inplace = True)

In [6]:
# create "income" dataframe and drop nulls
income = tables_df[['ano', 'mês', 'dia', 'receita', 'depositante']].copy()
income.dropna(subset = 'receita', axis = 0, inplace=True)
income.reset_index(drop = True, inplace = True)

In [7]:
# create categories for 'expenses'

mask = expenses['favorecido'].str.contains('sal|intercamb|alim|lanch|temperinho|rest|pão|café|panif|doce|confeitaria|subway|gratificação|sabor|hong ju|mansa', flags = re.I)
sal = np.where(mask, 'salário e auxílio alimentação', expenses['favorecido'])
expenses.loc[:,'favorecido'] = sal

mask2 = expenses['favorecido'].str.contains('sup|super|hiper|hipper|mercado|bistek|milium|embala|atacad|cassol|agua|distrib|loja|zeus|comerc', flags = re.I)
sup = np.where(mask2, 'suprimentos', expenses['favorecido'])
expenses.loc[:,'favorecido'] = sup

mask3 = expenses['favorecido'].str.contains('medic|clínica|clinica|vet|remédio|farm|biofilia|gral|asamed|pet|cuidados|panvel|acupuntura|raio x|praiana|castração|tosa|cremat', flags = re.I)
medic = np.where(mask3, 'medicações e veterinário', expenses['favorecido'])
expenses.loc[:,'favorecido'] = medic

mask4 = expenses['favorecido'].str.contains('hosp|poeta|lilian ribeiro|lar|cuidadora|casa|canil|canis|inse', flags = re.I)
hosp = np.where(mask4, 'hospedagem e manutenção de canis', expenses['favorecido'])
expenses.loc[:,'favorecido'] = hosp

mask5 = expenses['favorecido'].str.contains('agro|ração|reção|pecuária|terra|arcadia', flags = re.I)
rac = np.where(mask5, 'ração e agropecuária', expenses['favorecido'])
expenses.loc[:,'favorecido'] = rac

mask7 = expenses['favorecido'].str.contains('combust|escap|posto|seguro|veíc|motorista|car|borrack|park|auto|taxi|táxi|tàxi|frete|motor|estac|floripeças|multa|mecanica|mecânica|mecãnica|parachoque|borrach|estrela|localiza', flags = re.I)
trans = np.where(mask7, 'transporte', expenses['favorecido'])
expenses.loc[:,'favorecido'] = trans

mask6 = expenses['favorecido'].str.contains('camis|malhas|mat|jebelus|agenda|vest|patchwork|plumas|bazar|graf|gráf|digital|mtools|eletr|hd|site|notebook|face|cópias|copia|plot|mídia|chip|marca|mix|print', flags = re.I)
vend = np.where(mask6, 'bazar, divulgação e tecnologias', expenses['favorecido'])
expenses.loc[:,'favorecido'] = vend

mask8 = expenses['favorecido'].str.contains('empr|apli|renda', flags = re.I)
emp = np.where(mask8, 'empréstimos e aplicações', expenses['favorecido'])
expenses.loc[:,'favorecido'] = emp

mask9 = expenses['favorecido'].str.contains('tarif|INSS|IR|taxa|IPVA|DARF|celesc|tim|distrit|alvará|cadastro|licen|trib|certif|dss|cmf', flags = re.I)
tax = np.where(mask9, 'taxas, tarifas e impostos', expenses['favorecido'])
expenses.loc[:,'favorecido'] = tax

outros_mask = ~(expenses['favorecido'].isin(['salário e auxílio alimentação', 'suprimentos', 'medicações e veterinário', 'hospedagem e manutenção de canis', 'ração e agropecuária', 'bazar, divulgação e tecnologias', 'transporte', 'empréstimos e aplicações', 'taxas, tarifas e impostos']))
expenses.loc[outros_mask,'favorecido'] = "outros"
expenses['favorecido'].value_counts()

transporte                          1330
salário e auxílio alimentação       1025
taxas, tarifas e impostos            724
ração e agropecuária                 656
medicações e veterinário             536
outros                               321
hospedagem e manutenção de canis     244
suprimentos                          203
bazar, divulgação e tecnologias       82
Name: favorecido, dtype: int64

In [8]:
# create categories for income

mask = income['depositante'].str.contains('doaç|ração|v.d.|\d|DD|cielo|cielo|pag|pic', regex = True, flags = re.I)
doa = np.where(mask, 'doações e vendas', income['depositante'])
income.loc[:,'depositante'] = doa

mask2 = income['depositante'].str.contains('empr|apli|resg|rend', regex = True, flags = re.I)
emp = np.where(mask2, 'empréstimos e aplicações', income['depositante'])
income.loc[:,'depositante'] = emp

mask3 = income['depositante'].str.contains('Pista', regex = True, flags = re.I)
proj = np.where(mask3, 'projeto Auto Pista Litoral Sul', income['depositante'])
income.loc[:,'depositante'] = proj

mask4 = income['depositante'].str.contains('justiça|PMF ', regex = True, flags = re.I)
gov = np.where(mask4, 'governo', income['depositante'])
income.loc[:,'depositante'] = gov

mask5 = income['depositante'].str.contains('div|tran|int|seguro|rog|BB|dif', regex = True, flags = re.I)
out = np.where(mask5, 'outros', income['depositante'])
income.loc[:,'depositante'] = out

In [9]:
# final categories for 'expenses'
expenses['favorecido'].value_counts()

transporte                          1330
salário e auxílio alimentação       1025
taxas, tarifas e impostos            724
ração e agropecuária                 656
medicações e veterinário             536
outros                               321
hospedagem e manutenção de canis     244
suprimentos                          203
bazar, divulgação e tecnologias       82
Name: favorecido, dtype: int64

In [10]:
# final categories for 'income'
income['depositante'].value_counts()

doações e vendas                  1710
empréstimos e aplicações           411
projeto Auto Pista Litoral Sul      31
governo                             14
outros                              10
Name: depositante, dtype: int64

In [11]:
#converting data types
expenses['despesas'] = expenses['despesas'].astype(str).str.strip()
expenses['despesas'] = expenses['despesas'].str.replace(',', '')
expenses['despesas'] = expenses['despesas'].astype(float)
expenses['ano'] = expenses['ano'].astype(object)
income['receita'] = income['receita'].astype(str).str.strip()
income['receita'] = income['receita'].str.replace(',', '')
income['receita'] = income['receita'].astype(float)

In [12]:
# read "animals" csv
animals = pd.read_excel(r"C:\Users\Vanessa\Downloads\Projeto Final\animais_2016_2019.xlsx", index_col = None)

In [13]:
# rename columns
animals.columns = animals.columns.str.lower().str.strip()

# Exploratory analysis

In [14]:
expenses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5121 entries, 0 to 5120
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ano         5121 non-null   object 
 1   mês         5121 non-null   object 
 2   dia         5114 non-null   float64
 3   despesas    5121 non-null   float64
 4   favorecido  5121 non-null   object 
dtypes: float64(2), object(3)
memory usage: 200.2+ KB


In [15]:
income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2176 entries, 0 to 2175
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ano          2176 non-null   float64
 1   mês          2176 non-null   object 
 2   dia          2175 non-null   float64
 3   receita      2176 non-null   float64
 4   depositante  2176 non-null   object 
dtypes: float64(3), object(2)
memory usage: 85.1+ KB


In [16]:
expenses.head()

,ano,mês,dia,despesas,favorecido
0,2016.0,Sep,1.0,2000.00,salário e auxílio alimentação
1,2016.0,Sep,2.0,363.00,ração e agropecuária
2,2016.0,Sep,2.0,10.00,transporte
3,2016.0,Sep,2.0,143.57,transporte
4,2016.0,Sep,2.0,50.00,outros


In [17]:
income.head()

,ano,mês,dia,receita,depositante
0,2016.0,Sep,1.0,565.0,doações e vendas
1,2016.0,Sep,2.0,350.0,doações e vendas
2,2016.0,Sep,5.0,360.0,doações e vendas
3,2016.0,Sep,6.0,140.0,doações e vendas
4,2016.0,Sep,8.0,440.0,doações e vendas


# Querying for metrics

In [18]:
# Média de recursos e custos por animal assistido (2016-2019)- 
# Total histórico de animais assistidos/aldeia (2016-2019;
# Quantidade de zoonoses tratada pela OBA.

In [69]:
# custos totais para manter as atividades (2020-2022);
expenses20_22 = expenses[expenses['ano'] >= 2020]
expenses20_22[['despesas', 'ano']].groupby(by ='ano').sum()

,despesas
ano,
2020.0,166005.92
2021.0,204564.25
2022.0,209749.74


In [92]:
# média anual total de custos X recurso (2020-2022):
def anual_avg(x):
    return round(sum(x)/3, 2)
expenses20_22.groupby(['favorecido']).aggregate({'despesas': anual_avg}).sort_values(by = 'despesas', ascending = True)

,despesas
favorecido,
"bazar, divulgação e tecnologias",1386.58
suprimentos,1929.71
outros,4507.66
hospedagem e manutenção de canis,8271.22
transporte,24367.49
ração e agropecuária,29323.65
medicações e veterinário,36519.27
"taxas, tarifas e impostos",42384.97
salário e auxílio alimentação,44749.42


In [91]:
# média mensal total de custos X recurso (2020-2022):
def month_avg(x):
    return round(sum(x)/(3*12), 2)
expenses20_22.groupby(['favorecido']).aggregate({'despesas': month_avg}).sort_values(by = 'despesas', ascending = True)

,despesas
favorecido,
"bazar, divulgação e tecnologias",115.55
suprimentos,160.81
outros,375.64
hospedagem e manutenção de canis,689.27
transporte,2030.62
ração e agropecuária,2443.64
medicações e veterinário,3043.27
"taxas, tarifas e impostos",3532.08
salário e auxílio alimentação,3729.12


In [72]:
# custos/tipos de recurso (2020-2022);
expenses20_22.groupby(['ano', 'favorecido']).aggregate({'despesas':sum})

despesas
ano    favorecido                                
2020.0 bazar, divulgação e tecnologias    2360.00
       hospedagem e manutenção de canis   8571.03
       medicações e veterinário          32984.10
       outros                             5366.59
       ração e agropecuária              30919.37
       salário e auxílio alimentação     33707.01
       suprimentos                        1617.61
       taxas, tarifas e impostos         30577.50
       transporte                        19902.71
2021.0 bazar, divulgação e tecnologias     695.98
       hospedagem e manutenção de canis   8563.57
       medicações e veterinário          31298.24
       outros                             4171.41
       ração e agropecuária              29681.08
       salário e auxílio alimentação     55542.14
       suprimentos                        1717.74
       taxas, tarifas e impostos         49866.66
       transporte                        23027.43
2022.0 bazar, divulgação e tecnologias    1103.75
       hospedagem e manutenção de canis   7679.06
       medicações e veterinário          45275.47
       outros                             3984.99
       ração e agropecuária              27370.49
       salário e auxílio alimentação     44999.10
       suprimentos                        2453.78
       taxas, tarifas e impostos         46710.76
       transporte                        30172.34

In [ ]:
# total de animais 2022 / custos 2022